In [2]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
import warnings

# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")

In [3]:
MMRstatus_data=pd.read_csv('../../data/OriginalData/P53&MMR.csv',encoding='cp949')
def svsToTiff(imageCount):
    global svsWidth,svsHeight,path,scene,fileName
    slide_w_tile_size=788
    slide_h_tile_size=705
    for widthCount in range(0, int(svsWidth // slide_w_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_h_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_w_tile_size, heightCount* slide_h_tile_size])), path.contains_point(np.array([widthCount* slide_w_tile_size, (heightCount+1)* slide_h_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_w_tile_size, heightCount* slide_h_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_w_tile_size, (heightCount+1)* slide_h_tile_size]))])
            inside = np.mean(points)>0.2
            Full=np.mean(points)>0
            if (inside):
                image = scene.read_block((widthCount * slide_w_tile_size, heightCount * slide_h_tile_size, slide_w_tile_size, slide_h_tile_size),size=(256,256))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(256*256)/4:
                    TiffImageSave(image, fileName, imageCount)
                    imageCount += 1
            
    return imageCount
def TiffImageSave(image1,fileName,imageCount):
    global normalizer2x,normalizer5x,normalizer10x,tmp_img
    img=Image.fromarray(image1) 
    img.save('../../data/CycleGANData/standard/'+fileName+'_'+str(imageCount)+'.jpg')
    
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
    

In [4]:
svs_files=glob('../../data/OriginalData/svs/*.svs')

xml_files = [f.replace('/svs', '/xml') for f in svs_files]
xml_files = [f.replace('.svs', '.xml') for f in xml_files ]

In [5]:
for i in tqdm(range(len(svs_files))):
    try:
        file_extension_index=svs_files[i].find('.svs')
        fileName=os.path.basename(os.path.splitext(svs_files[i])[0])
        slide = slideio.open_slide(svs_files[i], "SVS")
        scene = slide.get_scene(0)
        svsWidth = scene.rect[2]
        svsHeight = scene.rect[3]
        imageCount=0
        magnification=80
        tile_size=256
        xml_open=xml_files[i]
        polygon_xml = ET.parse(xml_open)
        root = polygon_xml.getroot()
        root=list(root)[0]
        root=list(root)
        imageCount=0
        for annotation_count in range(len(root)):
            root1 =list(root[annotation_count])[0]
            polygon=[]
            for polygon_count in root1.iter('Coordinate'): 
                polygon.append([int(float(polygon_count.attrib['X'])),int(float(polygon_count.attrib['Y']))])
            path = mpltPath.Path(polygon)
            imageCount=svsToTiff(imageCount)
    except:
        file_extension_index=svs_files[i].find('.svs')
        fileName=svs_files[i][:file_extension_index]
        print(fileName)

  0%|          | 0/1239 [00:00<?, ?it/s]

JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling value

../../data/OriginalData/svs/S10-02060-1


JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].


In [ ]:
len(svs_files)